<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" />

# Cours TAL – Labo 5 : Le modèle word2vec et ses applications

**Objectifs**
Le but de ce labo est de comparer un modèle word2vec pré-entraîné avec deux modèles que vous
entraînerez vous-mêmes, sur deux corpus de tailles différentes. La comparaison se fera sur une
tâche de similarité mots et sur une tâche de raisonnement par analogie, en anglais. Vous utiliserez la librairie Gensim de calcul de similarités pour le TAL.

## 1. Tester et évaluer un modèle déjà entraîné sur Google News

Installez gensim, une librairie Python qui fournit des outils pour travailler avec Word2Vec (avec
conda ou avec pip). **Attention** : la dernière version 4.2.3 de gensim est incompatible avec la
librairie scipy version 1.13, donc il faut installer la version 1.12 de scipy ; la variable Path doit
contenir `C:\ProgramData\Miniconda3\Library\` et `C:\ProgramData\Miniconda3\Library\bin\.`


In [ ]:
!pip install gensim

Obtenez depuis gensim le modèle word2vec pré-entraîné sur le corpus Google News en
écrivant : `w2v_vectors = gensim.downloader.load("word2vec-google-news-300")`, ce qui
téléchargera le fichier la première fois

In [97]:
import gensim.downloader

#Default path is C:\Users\username\gensim-data
w2v_vectors = gensim.downloader.load("word2vec-google-news-300")

Après avoir téléchargé le modèle, vous pouvez utiliser ainsi votre copie locale :
`w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)`.

In [98]:
from gensim.models import KeyedVectors

w2v_vectors = KeyedVectors.load_word2vec_format("./corpus/GoogleNews-vectors-negative300.bin", binary=True)

#### a. Quelle place en mémoire occupe le processus du notebook avec les vecteurs de mots ?

Nous installant l'extension jupyter-server-resource-usage, nous avons pu observer que le kernel du notebook occupait 2.8 Go de mémoire vive après le chargement du modèle téléchargé en amont

![](./img/memory_usage.png)


#### b. Quelle est la dimension de l'espace vectoriel dans lequel les mots sont représentés ?



In [99]:
print(w2v_vectors.key_to_index.__len__()) #Nombre de clés = nombre de mots
print(w2v_vectors.vector_size) #Taille du vecteur pour chaque clé

3000000
300


Un `KeyedVector` est une structure semblable à un dictionnaire ayant comme clé un mot et comme valeur un vecteur. Dans notre cas, nous avons 3000000 clés chacune ayant un vecteur de 300 entrées.

#### c. Quelle est la taille du vocabulaire connu du modèle ? Veuillez afficher 5 mots anglais qui sont dans le vocabulaire et deux qui ne le sont pas.

In [100]:
voc_model = set(w2v_vectors.index_to_key)
result_in_voc = {"hello", "world", "computer", "science", "data"}.intersection(voc_model)
result_not_in_voc = {"crapulous", "manichaean"}.difference(voc_model)

print(f"Mots dans le vocabulaire {result_in_voc}") #5 mots
print(f"Mots pas dans le vocabulaire {result_not_in_voc}")
print(f"taille du vocabulaire {len(voc_model)}")

Mots dans le vocabulaire {'science', 'world', 'data', 'computer', 'hello'}
Mots pas dans le vocabulaire {'crapulous', 'manichaean'}
taille du vocabulaire 3000000


#### d. Quelle est la distance entre les mots rabbit et carrot ? Veuillez expliquer en une phrase comment on mesure les distances entre deux mots grâce à leurs vecteurs

In [101]:
print(f"Distance entre les mots: ", w2v_vectors.distance("rabbit", "carrot"))

Distance entre les mots:  0.63693568110466


La distance entre deux mots est mesurée par la similarité cosinus entre les vecteurs de ces mots. Plus la valeur est proche de 0, plus les mots sont similaires, plus la valeur est proche de 1, plus les mots sont différents.

#### e. Considérez au moins 5 paires de mots anglais, certains proches par leurs sens, d’autres plus éloignés. Pour chaque paire, calculez la distance entre les deux mots. Veuillez indiquer si les distances obtenues correspondent à vos intuitions sur la proximité des sens des mots.

In [102]:
pairs = [("cat", "dog"), ("cat", "car"), ("hot", "cold"), ("shoe", "journalist"), ("height", "high")]

for pair in pairs:
    print(f"Distance entre les mots {pair}: ", w2v_vectors.distance(pair[0], pair[1]))
    

Distance entre les mots ('cat', 'dog'):  0.23905426263809204
Distance entre les mots ('cat', 'car'):  0.7847181558609009
Distance entre les mots ('hot', 'cold'):  0.539786159992218
Distance entre les mots ('shoe', 'journalist'):  0.8940958231687546
Distance entre les mots ('height', 'high'):  0.8072148114442825


- **('cat', 'dog'):** Il s'agit de deux animaux de compagnie souvent mis en comparaison ou en opposition. Le score étant faible, il est cohérent avec notre intuition.
- **('cat', 'car'):** Ces deux mots sont proches au sens de l'orthographe, mais n'ont pas forcément un lien sémantique fort. On pourrait éventuellement supposer une distance faible si le système prenait en compte les erreurs typographiques, étant donné que T et R sont à côté l'un de l'autre sur un clavier QWERTY, mais ce n'est pas le cas. Le score étant élevé, il est cohérent avec notre intuition.
- **('hot', 'cold'):** Il s'agit de deux antonymes, ils possèdent donc un même sens sémantique (la température), mais leur nature d'antonymes pourrait également les éloigner. Le score est relativement moyen (~0.5) ce qui confirme notre intuition
- **('shoe', 'journalist'):** Ces deux mots n'ont rien en commun, on s'attend à un score élevé, ce qui est le cas.
- **('height', 'high'):** Ces deux mots sont similaires (l'un étant l'adjectif de l'autre), on s'attendrait à un score proche, voir moyen, mais le score est beaucoup plus élevé que notre intuition

#### f. Pouvez-vous trouver des mots de sens opposés mais qui sont proches selon le modèle ? Comment expliquez-vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?

In [103]:
print(f"Top 10 des mots les plus similaire: ", w2v_vectors.most_similar("good", topn=10))
print(f"Distance entre 'good' et 'bad': ", w2v_vectors.distance("good", "bad"))

Top 10 des mots les plus similaire:  [('great', 0.7291510105133057), ('bad', 0.7190051078796387), ('terrific', 0.6889115571975708), ('decent', 0.6837348341941833), ('nice', 0.6836092472076416), ('excellent', 0.6442928910255432), ('fantastic', 0.6407778263092041), ('better', 0.6120728850364685), ('solid', 0.5806034207344055), ('lousy', 0.5764203071594238)]
Distance entre 'good' et 'bad':  0.28099489212036133


Le mot "good" est proche de "bad" dans le modèle word2vec, ce qui est surprenant car ces deux mots sont des antonymes. Cela peut s'expliquer par le fait que les mots "good" et "bad" sont souvent utilisés dans des contextes similaires, par exemple dans des critiques de films ou de restaurants. C'est un défaut du modèle word2vec, car il ne prend pas en compte le sens des mots, mais seulement leur fréquence d'apparition dans un corpus de texte.

#### g. calculez le score du modèle word2vec sur les données WordSimilarity-353. Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [70]:
from gensim.test.utils import datapath

pearson, spearman, oov_ration = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))
print(pearson)
print(spearman)
print(oov_ration)

PearsonRResult(statistic=0.6238773483165634, pvalue=1.7963227018764457e-39)
SignificanceResult(statistic=0.6589215888009288, pvalue=2.5346056459149263e-45)
0.0


Le score de Pearson et le score de Spearman mesurent la corrélation entre les similarités de mots prédites par le modèle et les similarités de mots humaines (score de référence).
Le score de Pearson est une mesure de la corrélation linéaire entre deux variables, tandis que le score de Spearman est une mesure de la corrélation monotone. Un score élevé indique que le modèle prédit bien les similarités de mots humaines.
La pvalue est une valeur indiquant si le coefficient statistique est calculé par hasard. Plus cette valeur est faible, plus la corrélation statistique est significative (c.à.d. que la corrélation n'est pas due au hasard).

#### h. calculez le score du modèle word2vec sur les données questions-words.txt. Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [73]:
score, section = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'), dummy4unknown=True)
print(score)

0.7320405239459681


On évalue ici l'analogie du modèle par rapport à certains mots dans certaines catégories de la façon suivante "a est à b ce que c est à d". Par exemple: "Athène est à la Grèce ce que Paris est à la France". Le modèle doit donc trouver le mot manquant d dans la phrase. Le score est calculé en fonction du nombre de réponses correctes trouvées par le modèle.

# 2. Entraîner deux nouveaux modèles word2vec à partir de deux corpus

a. En utilisant gensim.downloader, récupérez le corpus qui contient les 10^8 premiers caractères de Wikipédia (en anglais) avec la commande : corpus = gensim.downloader.load('text8'). Combien de phrases et de mots (tokens) possède ce corpus ?

In [55]:
corpus = gensim.downloader.load('text8')
all_docs = [d for d in corpus]

In [56]:
count_docs = len(all_docs)
print(f"Nombre de phrases (documents) : {count_docs}")
nb_words = sum(len(l) for l in all_docs)
print(f"Nombre de tokens : {nb_words}")

Nombre de phrases (documents) : 1701
Nombre de tokens : 17005207


b. Entraînez un nouveau modèle word2vec sur ce nouveau corpus (voir la documentation de Word2vec). Si nécessaire, procédez progressivement, en commençant par utiliser 1% du corpus, puis 10%, etc., pour contrôler le temps que cela prend.
• Veuillez indiquer la dimension choisie pour le embedding de ce nouveau modèle.
• Combien de temps prend l’entraînement sur le corpus total ?
• Quelle est la taille (en Mo) du modèle word2vec résultant ?

In [76]:
import time
from gensim.models import Word2Vec
# Prend seulement x% des phrases du corpus
percentage = 100
docs_to_take = int((count_docs / (100 / percentage)))
docs = all_docs[:docs_to_take]

epochs = 50
dimensionality = 300

start = time.time()
model = Word2Vec(sentences=docs, 
                 vector_size=dimensionality, 
                 workers=12, 
                 min_count=1,
                 epochs=epochs)
end = time.time()
training_time_secs = round(end - start, 3)

def estimate(projected, current):
    return f"{round(training_time_secs * (projected / current), 3)}s"

print(f"Dimensionalité : {dimensionality}, epochs : {epochs}, pourcentage du corpus: {percentage}%")
print(f"Temps d'exécution : {training_time_secs}s")
print(f"Temps d'exécution estimé si 100% du corpus était utilisé : {estimate(100, percentage)}")

print(f"Temps d'exécution estimé si 10 epochs étaient utilisés : {estimate(10, epochs)}")
print(f"- si 25 epochs: {estimate(25, epochs)}")
print(f"- si 50 epochs : {estimate(50, epochs)}")
print(f"- si 100 epochs : {estimate(100, epochs)}")


full_estimated = round(training_time_secs * (100 / percentage) * (100 / epochs), 3)
print(f"Temps d'exécution estimé avec 100 epochs et 100% des documents : {full_estimated}s ")

# Avec 100% du corpus et 50 epochs, nous avons un temps d'exécution d'environ 13 minutes.


Dimensionalité : 300, epochs : 50, pourcentage du corpus: 100%
Temps d'exécution : 776.487s
Temps d'exécution estimé si 100% du corpus était utilisé : 776.487s
Temps d'exécution estimé si 10 epochs étaient utilisés : 155.297s
- si 25 epochs: 388.243s
- si 50 epochs : 776.487s
- si 100 epochs : 1552.974s
Temps d'exécution estimé avec 100 epochs et 100% des documents : 1552.974s 


In [79]:
model.save("text8_model.bin")

In [80]:
def print_est_mem(m):
    size_estimation = m.estimate_memory()["total"] / 1024 / 1024
    print(f"Estimation de la taille du modèle : {round(size_estimation, 3)} Mo")

print_est_mem(model)


Estimation de la taille du modèle : 702.073 Mo


c. Mesurez la qualité de ce modèle comme en (1g) et (1h). Ce modèle est-il meilleur que celui entraîné sur Google News ? Quelle est selon vous la raison de la différence ?

In [81]:
# Évaluation selon 1g

text8_w2v_vectors = model.wv
text8_pearson, text8_spearman, text8_oov_ration = text8_w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

print("=== Google News ===")
print(pearson)
print(spearman)
print(oov_ration)

print()
print("=== text8 ===")
print(text8_pearson)
print(text8_spearman)
print(text8_oov_ration)


=== Google News ===
PearsonRResult(statistic=0.6238773483165634, pvalue=1.7963227018764457e-39)
SignificanceResult(statistic=0.6589215888009288, pvalue=2.5346056459149263e-45)
0.0

=== text8 ===
PearsonRResult(statistic=0.624862501969741, pvalue=1.2590346689054365e-39)
SignificanceResult(statistic=0.6658576342425047, pvalue=1.4174213371582874e-46)
0.0


In [82]:
# Evaluation selon 1h (questions-words.txt)

text8_score, text8_section = text8_w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'), dummy4unknown=True)
print("Score de Google News : ", score)
print("Score de text8 : ", text8_score)




Score de Google News :  0.7320405239459681
Score de text8 :  0.31349774866966845


In [ ]:
# Sur les analogies, le score du modèle entrainé est très faible.
# Il peut y avoir plusieurs explications : pour commencer, il est possible que les hyper-paramètres sélectionnés pour Google News soient meilleurs. 
# Cependant, l'explication la plus plausible serait que les 10^8 caractères de Wikipedia forment des phrases incohérentes telles que "gate gate gated gates gateway gateway two zero zero zero gauss gaussian" ou "of b are orthogonal k e j zero for all k j in b with k j dense span the linear span of b is dense in h we", qui est du pseudo-code. Le modèle est donc biaisé car les sémantiques ne sont pas juste.
# Il est également précisé dans le repo https://github.com/piskvorky/gensim-data?tab=readme-ov-file que ce dataset devrait plutôt être utilisé pour les tests.

# Le dataset de Google News étant bien plus cohérent et varié, celui-ci produit de meilleurs résultats. 

d. Téléchargez maintenant le corpus quatre fois plus grand constitué de la concaténation du corpus text8 et des dépêches économiques de Reuters (413 Mo) fourni en ligne par l’enseignant et appelé wikipedia_augmented.dat. Entraînez un nouveau modèle word2vec sur ce corpus, en précisant aussi la dimension choisie pour le plongement (embedding).
• Utilisez la classe Text8Corpus() pour charger le corpus et pour faire la tokenisation et la segmentation en phrases.
• Combien de temps prend l’entraînement ?
• Quelle est la taille (en Mo) du modèle word2vec résultant ?

In [86]:
from gensim.models.word2vec import Text8Corpus
augmented_corpus = Text8Corpus("./corpus/wikipedia_augmented.dat")
aug_docs_all = [d for d in augmented_corpus]
aug_count_docs = len(aug_docs_all)


In [88]:
# Prend seulement x% des phrases du corpus augmenté
print("Entrainement du modèle augmenté ...")

aug_percentage = 100
aug_docs_to_take = int((aug_count_docs / (100 / aug_percentage)))
aug_docs = aug_docs_all[:aug_docs_to_take]

aug_epochs = 25
aug_dimensionality = 300

aug_start = time.time()
aug_model = Word2Vec(sentences=aug_docs,
                 vector_size=aug_dimensionality,
                 workers=12,
                 min_count=1,
                 epochs=aug_epochs)
aug_end = time.time()
aug_training_time_secs = round(aug_end - aug_start, 3)

def aug_estimate(projected, current):
    return f"{round(aug_training_time_secs * (projected / current), 3)}s"

print(f"Dimensionalité : {aug_dimensionality}, epochs : {aug_epochs}, pourcentage du corpus: {aug_percentage}%")
print(f"Temps d'exécution : {aug_training_time_secs}s")
print(f"Temps d'exécution estimé si 100% du corpus était utilisé : {aug_estimate(100, aug_percentage)}")

print(f"Temps d'exécution estimé si 10 epochs étaient utilisés : {aug_estimate(10, aug_epochs)}")
print(f"- si 25 epochs: {aug_estimate(25, aug_epochs)}")
print(f"- si 50 epochs : {aug_estimate(50, aug_epochs)}")
print(f"- si 100 epochs : {aug_estimate(100, aug_epochs)}")


aug_full_estimated = round(aug_training_time_secs * (100 / aug_percentage) * (100 / aug_epochs), 3)
print(f"Temps d'exécution estimé avec 100 epochs et 100% des documents : {aug_full_estimated}s ")

# Avec 100% du corpus et 25 epochs, nous avons un temps d'exécutions d'environ 25 minutes.



Entrainement du modèle augmenté ...
Dimensionalité : 300, epochs : 25, pourcentage du corpus: 100%
Temps d'exécution : 1564.783s
Temps d'exécution estimé si 100% du corpus était utilisé : 1564.783s
Temps d'exécution estimé si 10 epochs étaient utilisés : 312.957s
- si 25 epochs: 782.391s
- si 50 epochs : 1564.783s
- si 100 epochs : 3129.566s
Temps d'exécution estimé avec 100 epochs et 100% des documents : 6259.132s 


In [92]:
print_est_mem(aug_model)

Estimation de la taille du modèle : 1464.749 Mo


e. Testez ce modèle comme en (1g) et (1h). Est-il meilleur que le précédent ? Pour quelle raison ?

In [93]:
# Évaluation selon 1g
aug_text8_w2v_vectors = aug_model.wv
aug_text8_pearson, aug_text8_spearman, aug_text8_oov_ration = aug_text8_w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

print("=== text8 ===")
print(text8_pearson)
print(text8_spearman)
print(text8_oov_ration)


print("=== text8 augmenté ===")
print(aug_text8_pearson)
print(aug_text8_spearman)
print(aug_text8_oov_ration)


=== text8 ===
PearsonRResult(statistic=0.624862501969741, pvalue=1.2590346689054365e-39)
SignificanceResult(statistic=0.6658576342425047, pvalue=1.4174213371582874e-46)
0.0
=== text8 augmenté ===
PearsonRResult(statistic=0.5281846908809424, pvalue=9.395485951742592e-27)
SignificanceResult(statistic=0.549538782656594, pvalue=3.0449829069408257e-29)
0.0


In [96]:
# Evaluation selon 1f

aug_text8_score, aug_text8_section = aug_text8_w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'), dummy4unknown=True)
print("Score de text8 : ", text8_score)
print("Score de text8 augmenté : ", aug_text8_score)


Score de text8 :  0.31349774866966845
Score de text8 augmenté :  0.3905546459271388


In [ ]:
# Le score est légèrement meilleur car l'ajout des données de Reuters permet de contrebalancer les problèmes énoncés au point 2c. Il y a plus de phrases cohérentes et variées et, ainsi, le modèle est meilleur mais pas parfait. Il faudrait retirer la partie de Wikipédia et garder uniquement la partie de Reuters afin de confirmer ces dires, tout en tenant en compte le fait que Reuters est orienté économie alors que Google News est plus généraliste.